| 단계                                            | 목적                 | 이유                                 |
| --------------------------------------------- | ------------------ | ---------------------------------- |
| 1️⃣ **라이트 정제**<br>(Unicode NFKC, 공백·URL 제거 등) | 요약 입력을 깨끗하게        | KoBART가 HTML 잔여·줄바꿈·URL에 민감 → 품질 ↓ |
| 2️⃣ **1차 중복 제거**<br>(`link`·`title+datetime`) | 불필요한 인퍼런스 절감       | 요약·감성 분석 시간은 텍스트 건수에 비례            |
| 3️⃣ **KoBART 요약**                             | `summary` 컬럼 생성    | 이후 단계에서 짧은 텍스트 활용 → RAM·CPU 부담↓    |
| 4️⃣ **감성 분석**<br>(KB-ALBERT·FinBERT 등)        | `neg/neu/pos` 스코어  | 짧은 summary를 쓰면 분석 속도 2-3배 개선       |
| 5️⃣ **최종 CSV 저장**                             | `raw_text`까지 보존 권장 | 추후 디버깅·재학습 대비                      |


위 절차 중 현재 파일에서는 1,2,3번 수행

In [ ]:
# -*- coding: utf-8 -*-
"""
Naver Finance 뉴스 → 통합·정제·중복 제거 → news_clean_YYYYMMDD.csv
"""
import glob, re, unicodedata, pandas as pd
from collections import Counter

# ───────────────────────────────
# 1. 파일 로드
# ───────────────────────────────
def load_csvs(pattern="/Users/yujimin/KB AI CHALLENGE/project/results/*.csv") -> pd.DataFrame:
    files = glob.glob(pattern)
    print(f"📂 Found {len(files)} CSV file(s)")
    return pd.concat((pd.read_csv(f, encoding="utf-8") for f in files), ignore_index=True)

# ───────────────────────────────
# 2. 텍스트 정규화
# ───────────────────────────────
URL_RE = re.compile(r"https?://\S+")

def normalize(text: str) -> str:
    text = unicodedata.normalize("NFKC", str(text))
    text = URL_RE.sub("", text)
    return re.sub(r"\s+", " ", text).strip()

# ───────────────────────────────
# 3. 전처리 & 중복 제거
# ───────────────────────────────
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(subset=["title", "content"]).drop_duplicates(subset=["link"])

    df["title"]   = df["title"].apply(normalize)
    df["content"] = df["content"].apply(normalize)
    df["text"]    = df["title"] + " [SEP] " + df["content"]

    # (선택) 동일 제목·날짜 중복 제거 – 본문 길이 긴 기사 우선
    df["len"] = df["content"].str.len()
    df = (
        df.sort_values("len", ascending=False)
          .drop_duplicates(subset=["title", "datetime"], keep="first")
          .drop(columns="len")
    )
    return df[["stock_name", "datetime", "text", "link"]]

# ───────────────────────────────
# 4. 저장 파일명 결정 로직
# ───────────────────────────────
def decide_filename(df: pd.DataFrame, out_dir="/Users/yujimin/KB AI CHALLENGE/project") -> str:
    # datetime 컬럼에서 ‘YYYY.MM.DD’ 부분만 추출
    dates = df["datetime"].astype(str).str.extract(r"(\d{4}\.\d{2}\.\d{2})")[0]
    # 가장 이른 날짜(earliest) 선택 — 필요 시 latest/most common 으로 교체
    target = pd.to_datetime(dates, format="%Y.%m.%d").min().strftime("%Y%m%d")
    return f"{out_dir}/news_clean_{target}.csv"

# ───────────────────────────────
# 5. 메인
# ───────────────────────────────
if __name__ == "__main__":
    raw_df   = load_csvs()
    clean_df = preprocess(raw_df)
    print(f"✅ After cleaning: {clean_df.shape}")

    out_csv = decide_filename(clean_df)
    clean_df.to_csv(out_csv, index=False, encoding="utf-8")
    print(f"🎉 Saved → {out_csv}")

📂 Found 4 CSV file(s)
✅ After cleaning: (728, 4)
🎉 Saved → /Users/yujimin/KB AI CHALLENGE/project/news_clean_20250805.csv


### 뉴스 텍스트 요약

In [8]:
# 코랩

#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
news_clean_20250805.csv → KoBART 요약 → text 컬럼 제거 → news_summary_20250805.csv
"""

from __future__ import annotations
import math, sys
from pathlib import Path

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# ───────────────────────────────
# 1. 경로·하이퍼파라미터
# ───────────────────────────────
# IN_CSV  = "/content/drive/MyDrive/Invemotion/news_clean_20250805.csv"
# OUT_CSV = "/content/drive/MyDrive/Invemotion/news_summary_20250805.csv"
IN_CSV  = "/Users/yujimin/KB AI CHALLENGE/project/news_clean_20250805.csv"
OUT_CSV = "/Users/yujimin/KB AI CHALLENGE/project/news_summary_20250805.csv"


BATCH_SIZE      = 8          # GPU VRAM 4 GB 기준
MAX_INPUT_LEN   = 512
MAX_SUMMARY_LEN = 128
DEVICE          = "cuda" if torch.cuda.is_available() else "cpu"


# ───────────────────────────────
# 2. 모델 로드
# ───────────────────────────────
print("🔄  Loading KoBART summarizer …")
tok   = AutoTokenizer.from_pretrained("digit82/kobart-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("digit82/kobart-summarization").to(DEVICE)
model.eval()

🔄  Loading KoBART summarizer …


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_n

In [ ]:
# df = pd.read_csv(IN_CSV, encoding="utf-8")
# # print(f"📂 Loaded {len(df):,} rows")

# # 요약 돌리기 전에 한 번만 실행
# enc = tok(df["text"].tolist(), truncation=False, padding=False)
# df["tok_len"] = [len(x) for x in enc["input_ids"]]
# df["truncated"] = df["tok_len"] > MAX_INPUT_LEN
# print(df["truncated"].mean())  # 512토큰 초과 비율

0.49313186813186816


In [ ]:
from tqdm.auto import tqdm

CHUNK_TOKENS   = 400   # 각 조각 입력 길이 (512보다 여유있게)
OVERLAP_TOKENS = 80    # 조각 간 겹침(문맥 보존)

def chunk_by_tokens(text: str, chunk_tokens=CHUNK_TOKENS, overlap=OVERLAP_TOKENS) -> list[str]:
    ids = tok.encode(text, add_special_tokens=True)
    chunks = []
    step = max(1, chunk_tokens - overlap)
    for s in range(0, len(ids), step):
        piece = ids[s:s+chunk_tokens]
        if not piece:
            break
        chunks.append(tok.decode(piece, skip_special_tokens=True))
        if s + chunk_tokens >= len(ids):
            break
    return chunks

def summarize_long(text: str) -> str:
    parts = chunk_by_tokens(text)
    # 1차: 조각별 요약 (배치 처리)
    part_sums = []
    for i in range(0, len(parts), BATCH_SIZE):
        part_sums += summarize_batch(parts[i:i+BATCH_SIZE])
    # 2차: 요약들의 요약(메타 요약)
    final = summarize_batch([" ".join(part_sums)])[0]
    return final

# 1) 길이 측정
enc = tok(df["text"].tolist(), truncation=False, padding=False)
df["tok_len"]    = [len(x) for x in enc["input_ids"]]
df["truncated"]  = df["tok_len"] > MAX_INPUT_LEN

# 2) 요약 실행: 짧은 문서(≤512)는 기존 배치 요약, 긴 문서(>512)는 슬라이딩 윈도우
summaries = [""] * len(df)

# 2-a) 짧은 문서 일괄 처리
short_idx = df.index[~df["truncated"]].tolist()
for i in tqdm(range(0, len(short_idx), BATCH_SIZE), desc="Short docs"):
    batch_ids = short_idx[i:i+BATCH_SIZE]
    texts = df.loc[batch_ids, "text"].tolist()
    outs = summarize_batch(texts)
    for j, idx in enumerate(batch_ids):
        summaries[idx] = outs[j]

# 2-b) 긴 문서 개별 처리(조각 요약 → 메타 요약)
long_idx = df.index[df["truncated"]].tolist()
for idx in tqdm(long_idx, desc="Long docs"):
    summaries[idx] = summarize_long(df.at[idx, "text"])

# 3) 컬럼 반영 및 정리
df.insert(2, "summary", summaries)
df = df.drop(columns=["text", "tok_len", "truncated"])


In [3]:
# #!/usr/bin/env python
# # -*- coding: utf-8 -*-
# """
# news_clean_20250805.csv → KoBART 요약 → text 컬럼 제거 → news_summary_20250805.csv
# """

# from __future__ import annotations
# import math, sys
# from pathlib import Path

# import pandas as pd
# import torch
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# # ───────────────────────────────
# # 1. 경로·하이퍼파라미터
# # ───────────────────────────────
# IN_CSV  = "/content/drive/MyDrive/Invemotion/news_clean_20250805.csv"
# OUT_CSV = "/content/drive/MyDrive/Invemotion/news_summary_20250805.csv"

# BATCH_SIZE      = 8          # GPU VRAM 4 GB 기준
# MAX_INPUT_LEN   = 512
# MAX_SUMMARY_LEN = 128
# DEVICE          = "cuda" if torch.cuda.is_available() else "cpu"


# # ───────────────────────────────
# # 2. 모델 로드
# # ───────────────────────────────
# print("🔄  Loading KoBART summarizer …")
# tok   = AutoTokenizer.from_pretrained("digit82/kobart-summarization")
# model = AutoModelForSeq2SeqLM.from_pretrained("digit82/kobart-summarization").to(DEVICE)
# model.eval()

# ───────────────────────────
# 3. 요약 함수 수정 (핵심)
# ───────────────────────────
def summarize_batch(texts: list[str]) -> list[str]:
    enc = tok(
        texts,
        max_length=MAX_INPUT_LEN,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )

    # KoBART에는 token_type_ids 불필요 – 제거
    if "token_type_ids" in enc:
        enc.pop("token_type_ids")

    enc = {k: v.to(DEVICE) for k, v in enc.items()}

    with torch.no_grad():
        out = model.generate(
            **enc,
            max_length=MAX_SUMMARY_LEN,
            num_beams=4,
            early_stopping=True
        )
    return tok.batch_decode(out, skip_special_tokens=True)


# # ───────────────────────────────
# # 3. 데이터 로드
# # ───────────────────────────────
# df = pd.read_csv(IN_CSV, encoding="utf-8")
# print(f"📂 Loaded {len(df):,} rows")


# ───────────────────────────────
# 4. KoBART 요약 (배치 처리)
# ───────────────────────────────
summaries: list[str] = []
total_batches = math.ceil(len(df) / BATCH_SIZE)

for i in range(total_batches):
    batch_texts = df["text"].iloc[i * BATCH_SIZE:(i + 1) * BATCH_SIZE].tolist()
    summaries.extend(summarize_batch(batch_texts))

    if (i + 1) % 10 == 0 or i == total_batches - 1:
        processed = min((i + 1) * BATCH_SIZE, len(df))
        print(f"📝 Summarized {processed:,} / {len(df):,}")

df.insert(2, "summary", summaries)   # summary 컬럼 삽입
df = df.drop(columns="text")         # ✔️ text 컬럼 제거


# ───────────────────────────────
# 5. 저장
# ───────────────────────────────
out_path = Path(OUT_CSV)
out_path.parent.mkdir(parents=True, exist_ok=True)

df.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"🎉 Saved → {out_path}  ({len(df):,} rows, columns: {list(df.columns)})")


🔄  Loading KoBART summarizer …


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


📂 Loaded 728 rows


ValueError: The following `model_kwargs` are not used by the model: ['token_type_ids'] (note: typos in the generate arguments will also show up in this list)